In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from pandas import DataFrame
import scipy.stats as stats

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [ ]:
data_folder = "../data/DS_case_to_delete/DS Challange - Churn - Data/"

In [ ]:
for i in os.listdir(data_folder):
    print(i)

In [ ]:
df_vars = pd.read_csv(os.path.join(data_folder, "case_training_data.csv"))
df_target = pd.read_csv(os.path.join(data_folder, "case_training_output.csv"))

In [ ]:
pd.set_option('display.max_columns', 40)

In [ ]:
df_vars.head()

In [ ]:
df_target.head()

JOIN target and features by contract_id

In [ ]:
df = df_vars.merge(df_target, on='contract_id')

In [ ]:
df.columns

# Target

In [ ]:
target = "has_churned_over_next_3m"
df[target].value_counts().plot.bar()
plt.show()

# Variable Types

In [ ]:
cat_vars = [var for var in df.columns if df[var].dtype == 'O']
cat_vars.remove('contract_id')

In [ ]:
cat_vars

In [ ]:
# cast all variables as categorical
df[cat_vars] = df[cat_vars].astype('O')

In [ ]:
# now let's identify the numerical variables

num_vars = [
    var for var in df.columns if var not in cat_vars and var != target
]
num_vars.remove('contract_id')
num_vars.remove('campaing_code_elect')
# number of numerical variables
len(num_vars)

In [ ]:
num_vars

# Missing Values

In [ ]:
# make a list of the variables that contain missing values
vars_with_na = [var for var in df.columns if df[var].isnull().sum() > 0]

# determine percentage of missing values (expressed as decimals)
# and display the result ordered by % of missin data

df[vars_with_na].isnull().mean().sort_values(ascending=False)

In [ ]:
# df.drop('campaing_code_elect', axis=1, inplace=True)

In [ ]:
# now we can determine which variables, from those with missing data,
# are numerical and which are categorical

cat_na = [var for var in cat_vars if var in vars_with_na]
num_na = [var for var in num_vars if var in vars_with_na]

print('Number of categorical variables with na: ', len(cat_na))
print('Number of numerical variables with na: ', len(num_na))

In [ ]:
cat_na

In [ ]:
num_na

# Relationship between missing data and Sale Price

In [ ]:
def analyse_na_value(df: DataFrame, var: str):

    # copy of the dataframe, so that we do not override the original data
    # see the link for more details about pandas.copy()
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.copy.html
    df = df.copy()

    # let's make an interim variable that indicates 1 if the
    # observation was missing or 0 otherwise
    df[var] = np.where(df[var].isnull(), 1, 0)

    # let's compare the median SalePrice in the observations where data is missing
    # vs the observations where data is available

    # determine the median price in the groups 1 and 0,
    # and the standard deviation of the sale price,
    # and we capture the results in a temporary dataset
    tmp = df.groupby(var)[target].agg(['mean', 'std'])

    # plot into a bar graph
    tmp.plot(kind="barh", y="mean", legend=False,
             xerr="std", title=target, color='green')

    plt.show()

In [ ]:
# let's run the function on each variable with missing data

for var in vars_with_na:
    analyse_na_value(df, var)

# Numerical variables

In [ ]:
print('Number of numerical variables: ', len(num_vars))

# visualise the numerical variables
df[['contract_id'] + num_vars].head()

# Temporal Variables

In [ ]:
# list of variables that contain year information

year_vars = [var for var in df.columns if 'date' in var]

year_vars

In [ ]:
# let's explore the values of these temporal variables

for var in year_vars:
    print(var, df[var].unique())
    print()

# Discrete variables

In [ ]:
#  let's male a list of discrete variables
discrete_vars = [var for var in num_vars if len(
    df[var].unique()) < 20 and var not in year_vars]


print('Number of discrete variables: ', len(discrete_vars))

In [ ]:
# let's visualise the discrete variables

df[discrete_vars].head()

In [ ]:
df[[target]+discrete_vars].corr()

# Continuous variables

In [ ]:
# make list of continuous variables
cont_vars = [
    var for var in num_vars if var not in discrete_vars+year_vars]

print('Number of continuous variables: ', len(cont_vars))

In [ ]:
cont_vars

In [ ]:
# let's visualise the continuous variables

df[cont_vars].head()

In [ ]:
# lets plot histograms for all continuous variables

df[cont_vars].hist(bins=30, figsize=(15,15))
plt.show()

In [ ]:
# first make a list with the super skewed variables
# for later

skewed = [
    'elect_cons_at_12m', 'gas_cons_at_12m', 'elect_cons_last_month']

In [ ]:
cont_vars = [
    'elect_cons_at_12m', 'gas_cons_at_12m', 'elect_cons_last_month',
    'fcst_bill_baseline_elect_next_month',
 'fcst_bill_baseline_elect_cal_year',
 'fcst_bill_baseline_elect_12m',
 'fcst_consumption_elect_next_month',
 'fcst_consumption_elect_12m',
 'fcst_consumption_elect_cal_year',
 'fcst_bill_meter_rent_12m',
 'fcst_price_period_1',
 'fcst_price_period_2',
 'fcst_price_power_period_1',
 'total_net_margin',
 'subscribed_power'
]

Yeo-Johnson transformation

In [ ]:
# Let's go ahead and analyse the distributions of the variables
# after applying a yeo-johnson transformation

# temporary copy of the data
tmp = df.copy()

for var in cont_vars:

    # transform the variable - yeo-johsnon
    tmp[var], param = stats.yeojohnson(df[var])

    
# plot the histograms of the transformed variables
tmp[cont_vars].hist(bins=30, figsize=(15,15))
plt.show()

Logarithmic transformation

In [ ]:
# Let's go ahead and analyse the distributions of these variables
# after applying a logarithmic transformation

tmp = df.copy()

for var in cont_vars:

    # transform the variable with logarithm
    tmp[var] = np.log(df[var])
    
tmp[cont_vars].hist(bins=30)
plt.show()